In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
from torchvision import models
from PIL import Image

import os
import numpy as np

In [5]:
train_data = torchvision.datasets.ImageFolder('E:/jupyter_program/artist painting/artist_distribute',
                                            transform=transforms.ToTensor())

test_data = torchvision.datasets.ImageFolder('E:/jupyter_program/artist painting/test_set',
                                               transform=transforms.ToTensor())
                                           
#train_data = datasets.CIFAR10('data', train=True, download=True, transform=train_transforms)
#test_data = datasets.CIFAR10('data', train=False, download=True, transform=test_transforms)

n_train_examples = int(len(train_data)*0.9)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = torch.utils.data.random_split(train_data, [n_train_examples, n_valid_examples])

In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 1822
Number of validation examples: 203
Number of testing examples: 46


In [7]:
BATCH_SIZE = 64

train_iterator = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
valid_iterator = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE)
test_iterator = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [12]:
  # 定义网络结构
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1,padding=1),  # 初始化卷积核大小为3*3
            #nn.Dropout2d(0.1),
            nn.ReLU(),  # 使用relu激活
            nn.MaxPool2d(kernel_size=2),  # 采用最大池化2*2
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, 1, 1), 
            #nn.Dropout2d(0.1),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 1, 1), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2),
        )
        '''self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, 1, 1), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2),
        )'''
        #self.fc1 = nn.Linear(32*64*64, 512) 
        self.fc1 = nn.Linear(64*32*32, 512) 
        self.relu = nn.ReLU()
        self.output = nn.Linear(512, 11)  # 使用全连接输出
        #self.fc1 = nn.Linear(32*16*16, 11)  # 使用全连接输出
        #self.bn1 = nn.BatchNorm2d(16)
        #self.bn2 = nn.BatchNorm2d(32)
        #self.bn3 = nn.BatchNorm2d(512)
        #self.fc_bn = nn.BatchNorm1d(512)
        self.dropout = nn.Dropout(p=0.2)
          
    def forward(self, x):
        x = self.conv1(x)
        #x = self.bn1(x)
        x = self.conv2(x)
        #x = self.bn2(x)
        #x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        #x = self.fc_bn(x)
        x = self.relu(x)
        output = self.output(x)
        output = self.dropout(output)
        return output


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#create resnet18 model
model = models.resnet18(pretrained = False)
fc_features = model.fc.in_features
model.fc = nn.Linear(fc_features, 11)
model = model.to(device)

In [14]:
BATCH_NORM = True

model = CNN().to(device)

In [15]:
print(model)

CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=65536, out_features=512, bias=True)
  (relu): ReLU()
  (output): Linear(in_features=512, out_features=11, bias=True)
  (dropout): Dropout(p=0.2)
)


In [78]:
optimizer = optim.Adam(model.parameters(),lr = 0.001, weight_decay = 1e-8)

In [79]:
criterion = nn.CrossEntropyLoss()

In [80]:
def calculate_accuracy(fx, y):
    preds = fx.max(1, keepdim=True)[1]
    correct = preds.eq(y.view_as(preds)).sum()
    acc = correct.float()/preds.shape[0]
    return acc

In [81]:
def train(model, device, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        fx = model(x)
        
        loss = criterion(fx, y)
        
        acc = calculate_accuracy(fx, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [82]:
def evaluate(model, device, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for (x, y) in iterator:

            #x = x.to(device)
            #y = y.to(device)

            fx = model(x)

            loss = criterion(fx, y)

            acc = calculate_accuracy(fx, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [83]:
EPOCHS = 20
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'myVGG-artist.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, device, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, device, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% |')

| Epoch: 01 | Train Loss: 6.771 | Train Acc: 16.05% | Val. Loss: 3.278 | Val. Acc: 24.39% |
| Epoch: 02 | Train Loss: 2.795 | Train Acc: 17.95% | Val. Loss: 2.407 | Val. Acc: 17.07% |
| Epoch: 03 | Train Loss: 2.183 | Train Acc: 25.02% | Val. Loss: 1.942 | Val. Acc: 41.46% |
| Epoch: 04 | Train Loss: 1.850 | Train Acc: 38.07% | Val. Loss: 1.508 | Val. Acc: 43.90% |
| Epoch: 05 | Train Loss: 1.563 | Train Acc: 45.29% | Val. Loss: 1.339 | Val. Acc: 46.34% |
| Epoch: 06 | Train Loss: 1.275 | Train Acc: 54.57% | Val. Loss: 1.208 | Val. Acc: 43.90% |
| Epoch: 07 | Train Loss: 1.104 | Train Acc: 60.46% | Val. Loss: 1.434 | Val. Acc: 46.34% |
| Epoch: 08 | Train Loss: 1.080 | Train Acc: 59.47% | Val. Loss: 1.084 | Val. Acc: 51.22% |
| Epoch: 09 | Train Loss: 0.946 | Train Acc: 63.78% | Val. Loss: 0.996 | Val. Acc: 60.98% |
| Epoch: 10 | Train Loss: 0.734 | Train Acc: 73.44% | Val. Loss: 0.914 | Val. Acc: 58.54% |
| Epoch: 11 | Train Loss: 0.667 | Train Acc: 75.09% | Val. Loss: 1.003 | Val. Ac

In [84]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

test_loss, test_acc = evaluate(model, device, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:05.2f}% |')

| Test Loss: 1.286 | Test Acc: 56.52% |
